In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from keras.models import model_from_json
from skimage.transform import resize

Using TensorFlow backend.


In [15]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):
    
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    
    #Check basic image data
    if ds.PatientPosition not in ['PA','AP']:
        print("Invalid image view/position")
        return None, None
    
    if ds.Modality != 'DX':
        print("Invalid image type")
        return None, None
    
    if ds.BodyPartExamined != 'CHEST':
        print("Invalid body part")
        return None, None
    
    img = ds.pixel_array
    return img, ds.StudyDescription
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_size): 
    
    resize_img = resize(img, img_size) #resizing
    proc_img = (resize_img - resize_img.mean())/225 #Normalization
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, "r")
    loaded_model = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model)
    model.load_weights(weight_path)  
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    if model.predict(img) > thresh:
        prediction = "Positive for Pneumonia"
    else:
        prediction = "Negative for Pneumonia"
    
    return prediction 

In [19]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"  #path to saved model
weight_path = "xray_class_my_model.best.hdf5" #path to saved best weights

img_size=(1,224,224,3) # This might be different if you did not use vgg16

#img_mean = np.mean(img)# loads the mean image value they used during training preprocessing
#img_std = 255 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.8 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img, label = check_dicom(i) 
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print('Model prediction: {} vs True Label: {}'.format(pred, label))

Load file test1.dcm ...
Model prediction: Negative for Pneumonia vs True Label: No Finding
Load file test2.dcm ...
Model prediction: Negative for Pneumonia vs True Label: Cardiomegaly
Load file test3.dcm ...
Model prediction: Negative for Pneumonia vs True Label: Effusion
Load file test4.dcm ...
Invalid body part
Load file test5.dcm ...
Invalid image type
Load file test6.dcm ...
Invalid image view/position
